In [2]:
import re
import pandas as pd
import json

In [6]:
with open("tourist_attractions.json", 'r') as f:
    data = json.load(f)
    
pd.DataFrame(data).to_excel("tourist_attractions.xlsx")

# Tourist Attraction Preprocessing

In [2]:
# with open("tourist_attractions_preprop_temp.json", 'w') as f:
#     json.dump(data, f)

In [11]:
with open("tourist_attractions_preprop_temp.json", 'r') as f:
    data = json.load(f)

data_temp = data.copy()

In [12]:
pd.DataFrame(data).head(3)

,data_id,place_id,place_name,rating,operating_hours,reviews,address,phone,website,reviews_link,photos_link,gps_coordinates,thumbnail,time_spent
0,0x2e7a5742a17e8c33:0xf92dc9a55003beb0,ChIJM4x-oUJXei4RsL4DUKXJLfk,MALIOBORO JOGJAKARTA,4.8,"{'friday': [[0, 1], [24, 0]], 'saturday': [[0,...",39786.0,"Jl. Malioboro, Sosromenduran, Gedong Tengen, K...",+62 856-5506-2240,https://malioborojogja.com/,https://serpapi.com/search.json?data_id=0x2e7a...,https://serpapi.com/search.json?data_id=0x2e7a...,"{'latitude': -7.7925764, 'longitude': 110.365843}",https://lh5.googleusercontent.com/p/AF1QipPu2E...,People typically spend up to 1.5 hours here
1,0x2e7a578819c338f1:0x72aa3ad97ecccfb6,ChIJ8TjDGYhXei4Rts_Mftk6qnI,Tourism Zone Malioboro,4.7,"{'friday': [[0, 1], [24, 0]], 'saturday': [[0,...",4967.0,"Jl. Margo Mulyo Jl. Jend. Ahmad Yani No.106, N...",None,None,https://serpapi.com/search.json?data_id=0x2e7a...,https://serpapi.com/search.json?data_id=0x2e7a...,"{'latitude': -7.7982487, 'longitude': 110.3652...",https://lh5.googleusercontent.com/p/AF1QipMmOC...,People typically spend up to 1 hour here
2,0x2e7a5796f097cf41:0x89fc8d46097f458b,ChIJQc-X8JZXei4Ri0V_CUaN_Ik,Plengkung Gading,4.6,"{'friday': [[0, 1], [24, 0]], 'saturday': [[0,...",2854.0,"Jl. Patehan Kidul No.4, Patehan, Kraton, Yogya...",None,None,https://serpapi.com/search.json?data_id=0x2e7a...,https://serpapi.com/search.json?data_id=0x2e7a...,"{'latitude': -7.8137278, 'longitude': 110.3629...",https://lh5.googleusercontent.com/p/AF1QipNfXm...,People typically spend up to 30 min here


In [13]:
# time_list_temp = list()
# data_err_idx = list()
# for idx, item in enumerate(data_temp):
#     try:
#         if isinstance(item['operating_hours'], str):
#             item['operating_hours'] = json.loads(item['operating_hours'].replace("\'", "\""))
#             data[idx]['operating_hours'] = item['operating_hours']
#         for k, v in item['operating_hours'].items():
#             data[idx]['operating_hours'][k] = v.encode('ascii', "ignore")
#             # if v not in time_list_temp:
#             #     time_list_temp.append(v)     
#     except:
#         data_err_idx.append(idx)

In [17]:
time_spent_list_temp = list()
data_err_idx = list()
for idx, item in enumerate(data_temp):
    try:
        # if isinstance(item['operating_hours'], str):
        #     item['operating_hours'] = json.loads(item['operating_hours'].replace("\'", "\""))
        # for v in item['operating_hours'].values():
        #     if v not in time_list_temp:
        #         time_list_temp.append(v)     
        if item['time_spent'] not in time_spent_list_temp:
            time_spent_list_temp.append(item['time_spent'])  
    except:
        data_err_idx.append(idx)

In [18]:
data_err_idx

[]

In [19]:
time_spent_list_temp

['People typically spend up to 1.5 hours here',
 'People typically spend up to 1 hour here',
 'People typically spend up to 30 min here',
 'People typically spend up to 45 min here',
 'People typically spend up to 2 hours here',
 'People typically spend 15 min to 1 hr here',
 'People typically spend 15-45 min here',
 'People typically spend 15 min here',
 'People typically spend up to 20 min here',
 'People typically spend 1.5-2.5 hours here',
 'People typically spend 30 min to 1.5 hr here',
 'People typically spend 25 min here',
 'People typically spend up to 25 min here',
 'People typically spend 45 min here',
 'People typically spend up to 1 hr here',
 'People typically spend 1.5 hr here',
 'People typically spend 1 hr to 2 hr here',
 'People typically spend 30 min to 1 hr here',
 'People typically spend 15 min to 30 min here',
 'People typically spend up to 1 hours here',
 'People typically spend up to 1.5-2.5 hours here',
 'People typically 15-45 min here',
 'People typically spen

In [34]:
def prep_regex(text):
    text = text.lower()
    text = text.replace("to", '-')
    text = re.sub("[A-Za-z\s]", "", text)
    if text.find('-') != -1:
        text = text.split('-')
        text = text[-1] 
    
    if text.find('.') != -1 or len(text) == 1:
        if text.find('.') != -1:
            text = text.split('.')
            text[0] = int(text[0]) * 60
            text[1] = 60 / (10 / int(text[1]))
            text = sum(text)
        else:
            text = int(text[0]) * 60
    return text
    
prep_regex("People typically spend up to 1.5-2.5 hours here")

150.0

In [35]:
for idx, item in enumerate(data_temp):
    try:
        data[idx]['time_spent'] = prep_regex(item['time_spent'])
    except:
        print(f"Data Error idx:{idx}-{item['time_spent']}")

In [36]:
data

[{'data_id': '0x2e7a5742a17e8c33:0xf92dc9a55003beb0',
  'place_id': 'ChIJM4x-oUJXei4RsL4DUKXJLfk',
  'place_name': 'MALIOBORO JOGJAKARTA',
  'rating': 4.8,
  'operating_hours': {'friday': [[0, 1], [24, 0]],
   'saturday': [[0, 1], [24, 0]],
   'sunday': [[0, 1], [24, 0]],
   'monday': [[0, 1], [24, 0]],
   'tuesday': [[0, 1], [24, 0]],
   'wednesday': [[0, 1], [24, 0]],
   'thursday': [[0, 1], [24, 0]]},
  'reviews': 39786.0,
  'address': 'Jl. Malioboro, Sosromenduran, Gedong Tengen, Kota Yogyakarta, Daerah Istimewa Yogyakarta 55271, Indonesia',
  'phone': '+62 856-5506-2240',
  'website': 'https://malioborojogja.com/',
  'reviews_link': 'https://serpapi.com/search.json?data_id=0x2e7a5742a17e8c33%3A0xf92dc9a55003beb0&engine=google_maps_reviews&hl=en',
  'photos_link': 'https://serpapi.com/search.json?data_id=0x2e7a5742a17e8c33%3A0xf92dc9a55003beb0&engine=google_maps_photos&hl=en',
  'gps_coordinates': {'latitude': -7.7925764, 'longitude': 110.365843},
  'thumbnail': 'https://lh5.google

In [206]:
with open("tourist_attractions_preprop_temp.json", 'w') as f:
    json.dump(data, f)

# Temp

In [10]:
with open("tourist_attractions.csv", 'r') as f:
    data_frame = pd.read_csv(f)
    
with open("tourist_attractions.json", 'r') as f:
    data_json = json.load(f)

In [15]:
data_frame

,ï»¿data_id,place_id,place_name,rating,operating_hours,reviews,address,phone,website,reviews_link,photos_link,gps_coordinates,thumbnail,time_spent,tarif_weekday,tarif_weekend
0,0x2e7a5742a17e8c33:0xf92dc9a55003beb0,ChIJM4x-oUJXei4RsL4DUKXJLfk,MALIOBORO JOGJAKARTA,4.8,"{'friday': [[0, 1], [24, 0]], 'saturday': [[0,...",39786.0,"Jl. Malioboro, Sosromenduran, Gedong Tengen, K...",+62 856-5506-2240,https://malioborojogja.com/,https://serpapi.com/search.json?data_id=0x2e7a...,https://serpapi.com/search.json?data_id=0x2e7a...,"{'latitude': -7.7925764, 'longitude': 110.365843}",https://lh5.googleusercontent.com/p/AF1QipPu2E...,90,0,0.0
1,0x2e7a578819c338f1:0x72aa3ad97ecccfb6,ChIJ8TjDGYhXei4Rts_Mftk6qnI,Tourism Zone Malioboro,4.7,"{'friday': [[0, 1], [24, 0]], 'saturday': [[0,...",4967.0,"Jl. Margo Mulyo Jl. Jend. Ahmad Yani No.106, N...",NaN,NaN,https://serpapi.com/search.json?data_id=0x2e7a...,https://serpapi.com/search.json?data_id=0x2e7a...,"{'latitude': -7.7982487, 'longitude': 110.3652...",https://lh5.googleusercontent.com/p/AF1QipMmOC...,60,0,0.0
2,0x2e7a5796f097cf41:0x89fc8d46097f458b,ChIJQc-X8JZXei4Ri0V_CUaN_Ik,Plengkung Gading,4.6,"{'friday': [[0, 1], [24, 0]], 'saturday': [[0,...",2854.0,"Jl. Patehan Kidul No.4, Patehan, Kraton, Yogya...",NaN,NaN,https://serpapi.com/search.json?data_id=0x2e7a...,https://serpapi.com/search.json?data_id=0x2e7a...,"{'latitude': -7.8137278, 'longitude': 110.3629...",https://lh5.googleusercontent.com/p/AF1QipNfXm...,30,0,0.0
3,0x2e7a5796d474c38b:0xa29ec3628bcfcdc0,ChIJi8N01JZXei4RwM3Pi2LDnqI,Bringin Kembar Yogyakarta,4.6,"{'friday': [[0, 1], [24, 0]], 'saturday': [[0,...",373.0,"59Q7+67G, Patehan, Kraton, Yogyakarta City, Sp...",NaN,NaN,https://serpapi.com/search.json?data_id=0x2e7a...,https://serpapi.com/search.json?data_id=0x2e7a...,"{'latitude': -7.8119357, 'longitude': 110.363194}",https://lh5.googleusercontent.com/p/AF1QipM3LI...,90,0,0.0
4,0x2e7a59a8c1161f4d:0xaca66c8a28a3d11c,ChIJTR8WwahZei4RHNGjKIpspqw,Rumah Hantu Malioboro,4.5,"{'friday': [[16, 0], [18, 0]], 'saturday': [[1...",573.0,"6938+P4X, Jl. Malioboro, Sosromenduran, Gedong...",NaN,http://www.instagram.com/rumahhantumalioboro,https://serpapi.com/search.json?data_id=0x2e7a...,https://serpapi.com/search.json?data_id=0x2e7a...,"{'latitude': -7.7956378, 'longitude': 110.3653...",https://lh5.googleusercontent.com/p/AF1QipPQR5...,45,25000,35000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,0x2e7a5717e66f25c9:0xc6314bc336c4b3e4,ChIJySVv5hdXei4R5LPENsNLMcY,Kampung Wisata Basen,4.3,"{'friday': [[8, 0], [18, 0]], 'saturday': [[8,...",245.0,"Jl. Kemasan No.53, Purbayan, Kotagede, Yogyaka...",NaN,NaN,https://serpapi.com/search.json?data_id=0x2e7a...,https://serpapi.com/search.json?data_id=0x2e7a...,"{'latitude': -7.824373899999999, 'longitude': ...",https://lh5.googleusercontent.com/p/AF1QipO6u7...,45,0,0.0
98,0x2e7a5796ef87e2f9:0xe1c66b49da06029d,ChIJ-eKH75ZXei4RnQIG2klrxuE,Yogyakarta Southern Square,4.7,"{'friday': [[0, 1], [24, 0]], 'saturday': [[0,...",2253.0,"59Q7+87Q, Jl. Gading, Patehan, Kraton, Yogyaka...",NaN,http://www.kratonjogja.id/,https://serpapi.com/search.json?data_id=0x2e7a...,https://serpapi.com/search.json?data_id=0x2e7a...,"{'latitude': -7.8116718999999994, 'longitude':...",https://lh5.googleusercontent.com/p/AF1QipPCu_...,120,0,0.0
99,0x2e7af83332b75f9f:0xbd35ada67229d3ed,ChIJn1-3MjP4ei4R7dMpcqatNb0,CitraGrand Mutiara Waterpark Yogyakarta,4.3,"{'friday': [[14, 0], [18, 0]], 'saturday': [[8...",2430.0,"Perumahan Citra Grand Mutiara, Jl. Wates, Pere...",+62 274 2820788,NaN,https://serpapi.com/search.json?data_id=0x2e7a...,https://serpapi.com/search.json?data_id=0x2e7a...,"{'latitude': -7.813734299999999, 'longitude': ...",https://lh5.googleusercontent.com/p/AF1QipO0Uz...,90,35000,55000.0
100,0x2e7a57bd92e2ae31:0xdc67cc53091bd2bf,ChIJMa7ikr1Xei4Rv9IbCVPMZ9w,Cemeti - Institute for Art and Society,4.6,"{'friday': [[10, 0], [17, 0]], 'saturday': [[1...",188.0,"Jl. DI Panjaitan No.41, Mantrijeron, Yog

In [16]:
data_frame['time_spent'] = [i*60 for i in data_frame['time_spent']]

In [31]:
data_frame

,ï»¿data_id,place_id,place_name,rating,operating_hours,reviews,address,phone,website,reviews_link,photos_link,gps_coordinates,thumbnail,time_spent,tarif_weekday,tarif_weekend
0,0x2e7a5742a17e8c33:0xf92dc9a55003beb0,ChIJM4x-oUJXei4RsL4DUKXJLfk,MALIOBORO JOGJAKARTA,4.8,"{'friday': [[0, 1], [24, 0]], 'saturday': [[0,...",39786.0,"Jl. Malioboro, Sosromenduran, Gedong Tengen, K...",+62 856-5506-2240,https://malioborojogja.com/,https://serpapi.com/search.json?data_id=0x2e7a...,https://serpapi.com/search.json?data_id=0x2e7a...,"{'latitude': -7.7925764, 'longitude': 110.365843}",https://lh5.googleusercontent.com/p/AF1QipPu2E...,5400,0,0.0
1,0x2e7a578819c338f1:0x72aa3ad97ecccfb6,ChIJ8TjDGYhXei4Rts_Mftk6qnI,Tourism Zone Malioboro,4.7,"{'friday': [[0, 1], [24, 0]], 'saturday': [[0,...",4967.0,"Jl. Margo Mulyo Jl. Jend. Ahmad Yani No.106, N...",NaN,NaN,https://serpapi.com/search.json?data_id=0x2e7a...,https://serpapi.com/search.json?data_id=0x2e7a...,"{'latitude': -7.7982487, 'longitude': 110.3652...",https://lh5.googleusercontent.com/p/AF1QipMmOC...,3600,0,0.0
2,0x2e7a5796f097cf41:0x89fc8d46097f458b,ChIJQc-X8JZXei4Ri0V_CUaN_Ik,Plengkung Gading,4.6,"{'friday': [[0, 1], [24, 0]], 'saturday': [[0,...",2854.0,"Jl. Patehan Kidul No.4, Patehan, Kraton, Yogya...",NaN,NaN,https://serpapi.com/search.json?data_id=0x2e7a...,https://serpapi.com/search.json?data_id=0x2e7a...,"{'latitude': -7.8137278, 'longitude': 110.3629...",https://lh5.googleusercontent.com/p/AF1QipNfXm...,1800,0,0.0
3,0x2e7a5796d474c38b:0xa29ec3628bcfcdc0,ChIJi8N01JZXei4RwM3Pi2LDnqI,Bringin Kembar Yogyakarta,4.6,"{'friday': [[0, 1], [24, 0]], 'saturday': [[0,...",373.0,"59Q7+67G, Patehan, Kraton, Yogyakarta City, Sp...",NaN,NaN,https://serpapi.com/search.json?data_id=0x2e7a...,https://serpapi.com/search.json?data_id=0x2e7a...,"{'latitude': -7.8119357, 'longitude': 110.363194}",https://lh5.googleusercontent.com/p/AF1QipM3LI...,5400,0,0.0
4,0x2e7a59a8c1161f4d:0xaca66c8a28a3d11c,ChIJTR8WwahZei4RHNGjKIpspqw,Rumah Hantu Malioboro,4.5,"{'friday': [[16, 0], [18, 0]], 'saturday': [[1...",573.0,"6938+P4X, Jl. Malioboro, Sosromenduran, Gedong...",NaN,http://www.instagram.com/rumahhantumalioboro,https://serpapi.com/search.json?data_id=0x2e7a...,https://serpapi.com/search.json?data_id=0x2e7a...,"{'latitude': -7.7956378, 'longitude': 110.3653...",https://lh5.googleusercontent.com/p/AF1QipPQR5...,2700,25000,35000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,0x2e7a5717e66f25c9:0xc6314bc336c4b3e4,ChIJySVv5hdXei4R5LPENsNLMcY,Kampung Wisata Basen,4.3,"{'friday': [[8, 0], [18, 0]], 'saturday': [[8,...",245.0,"Jl. Kemasan No.53, Purbayan, Kotagede, Yogyaka...",NaN,NaN,https://serpapi.com/search.json?data_id=0x2e7a...,https://serpapi.com/search.json?data_id=0x2e7a...,"{'latitude': -7.824373899999999, 'longitude': ...",https://lh5.googleusercontent.com/p/AF1QipO6u7...,2700,0,0.0
98,0x2e7a5796ef87e2f9:0xe1c66b49da06029d,ChIJ-eKH75ZXei4RnQIG2klrxuE,Yogyakarta Southern Square,4.7,"{'friday': [[0, 1], [24, 0]], 'saturday': [[0,...",2253.0,"59Q7+87Q, Jl. Gading, Patehan, Kraton, Yogyaka...",NaN,http://www.kratonjogja.id/,https://serpapi.com/search.json?data_id=0x2e7a...,https://serpapi.com/search.json?data_id=0x2e7a...,"{'latitude': -7.8116718999999994, 'longitude':...",https://lh5.googleusercontent.com/p/AF1QipPCu_...,7200,0,0.0
99,0x2e7af83332b75f9f:0xbd35ada67229d3ed,ChIJn1-3MjP4ei4R7dMpcqatNb0,CitraGrand Mutiara Waterpark Yogyakarta,4.3,"{'friday': [[14, 0], [18, 0]], 'saturday': [[8...",2430.0,"Perumahan Citra Grand Mutiara, Jl. Wates, Pere...",+62 274 2820788,NaN,https://serpapi.com/search.json?data_id=0x2e7a...,https://serpapi.com/search.json?data_id=0x2e7a...,"{'latitude': -7.813734299999999, 'longitude': ...",https://lh5.googleusercontent.com/p/AF1QipO0Uz...,5400,35000,55000.0
100,0x2e7a57bd92e2ae31:0xdc67cc53091bd2bf,ChIJMa7ikr1Xei4Rv9IbCVPMZ9w,Cemeti - Institute for Art and Society,4.6,"{'friday': [[10, 0], [17, 0]], 'saturday': [[1...",188.0,"Jl. DI Panjaitan No.41, M

In [34]:
for i, _ in enumerate(data_json):
    data_json[i]['time_spent'] = int(data_frame['time_spent'][i])
    data_json[i]['tarif_weekday'] = int(data_frame['tarif_weekday'][i])
    data_json[i]['tarif_weekend'] = int(data_frame['tarif_weekend'][i])
    


In [35]:
data_json

[{'data_id': '0x2e7a5742a17e8c33:0xf92dc9a55003beb0',
  'place_id': 'ChIJM4x-oUJXei4RsL4DUKXJLfk',
  'place_name': 'MALIOBORO JOGJAKARTA',
  'rating': 4.8,
  'operating_hours': {'friday': [[0, 1], [24, 0]],
   'saturday': [[0, 1], [24, 0]],
   'sunday': [[0, 1], [24, 0]],
   'monday': [[0, 1], [24, 0]],
   'tuesday': [[0, 1], [24, 0]],
   'wednesday': [[0, 1], [24, 0]],
   'thursday': [[0, 1], [24, 0]]},
  'reviews': 39786.0,
  'address': 'Jl. Malioboro, Sosromenduran, Gedong Tengen, Kota Yogyakarta, Daerah Istimewa Yogyakarta 55271, Indonesia',
  'phone': '+62 856-5506-2240',
  'website': 'https://malioborojogja.com/',
  'reviews_link': 'https://serpapi.com/search.json?data_id=0x2e7a5742a17e8c33%3A0xf92dc9a55003beb0&engine=google_maps_reviews&hl=en',
  'photos_link': 'https://serpapi.com/search.json?data_id=0x2e7a5742a17e8c33%3A0xf92dc9a55003beb0&engine=google_maps_photos&hl=en',
  'gps_coordinates': {'latitude': -7.7925764, 'longitude': 110.365843},
  'thumbnail': 'https://lh5.google

In [36]:
with open("tourist_attractions.json", 'w') as f:
    json.dump(data_json, f)